In [3]:
import pandas as pd
data = pd.read_csv("E_Vo_withDatafromPeriodicTable.csv")
df= pd.DataFrame(data)

In [4]:
df.keys()

Index(['Chemical formula', 'A', 'B', 'In literature', 'Valence A', 'Valence B',
       'Radius A [ang]', 'Radius B [ang]', 'Lowest distortion',
       'Formation energy [eV/atom]', 'Stability [eV/atom]',
       'Magnetic moment [mu_B]', 'Volume per atom [A^3/atom]', 'Band gap [eV]',
       'a [ang]', 'b [ang]', 'c [ang]', 'alpha [deg]', 'beta [deg]',
       'gamma [deg]', 'A-AtomicNumber', 'A-AtomicMass', 'A-NumberofNeutrons',
       'A-NumberofProtons', 'A-NumberofElectrons', 'A-Period', 'A-Group',
       'A-AtomicRadius', 'A-Electronegativity', 'A-FirstIonization',
       'A-Density', 'A-MeltingPoint', 'A-BoilingPoint', 'A-NumberOfIsotopes',
       'A-SpecificHeat', 'A-NumberofShells', 'B-AtomicNumber', 'B-AtomicMass',
       'B-NumberofNeutrons', 'B-NumberofProtons', 'B-NumberofElectrons',
       'B-Period', 'B-Group', 'B-Electronegativity', 'B-FirstIonization',
       'B-Density', 'B-MeltingPoint', 'B-BoilingPoint', 'B-NumberOfIsotopes',
       'B-SpecificHeat', 'B-NumberofShells',

In [5]:
df_temp1= [df['B'] == 'Fe']
df_temp2= [df['B'] == 'Co']
name = ['Fe','Co']
df_temp = df[df['B'].isin(name)]
df_temp = df_temp[['Radius A [ang]', 'A-Density','A-AtomicNumber','A-SpecificHeat',
                   'B-AtomicMass','Vacancy energy [eV/O atom]']]

In [6]:
import numpy as np

df_temp = df_temp.replace(to_replace='-',value=np.nan)
display(df_temp.info())
df_temp = df_temp.dropna()
df_temp = df_temp.astype(float)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 876 to 1459
Data columns (total 6 columns):
Radius A [ang]                146 non-null float64
A-Density                     146 non-null float64
A-AtomicNumber                146 non-null int64
A-SpecificHeat                136 non-null float64
B-AtomicMass                  146 non-null float64
Vacancy energy [eV/O atom]    124 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 8.0+ KB


None

In [ ]:
display(df_temp.head())
display(df_temp.describe())

In [ ]:
X = df_temp.values[:, 0:-1]
Y = df_temp.values[:,-1]
print(Y)

In [ ]:
import seaborn as sns
display(sns.pairplot(df_temp))
import matplotlib.pyplot as plt
plt.savefig("pairplot_features_selected.png")

In [ ]:
display(X.shape)
display(Y.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
#scaler  = MinMaxScaler()
X = scaler.fit_transform(X)

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
clf = SVR(gamma=0.1, kernel='rbf',C = 5)
clf.fit(X_train, Y_train) 
Y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(Y_pred,Y_test))
plt.scatter(X_test[:,0], Y_test)
plt.scatter(X_test[:,0], Y_pred)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=3)
knn.fit(X_train, Y_train) 
Y_pred = knn.predict(X_test)
print(mean_squared_error(Y_pred,Y_test))
plt.scatter(X_test[:, 0], Y_test)
plt.scatter(X_test[:, 0], Y_pred)

In [ ]:
#nn model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import optimizers
shape = X_train.shape

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(10, kernel_initializer='normal',input_dim =shape[1], activation='tanh'))

# The Hidden Layers :
NN_model.add(Dense(10, kernel_initializer='normal',activation='tanh'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adagrad', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
# history = NN_model.fit(X,Y,epochs=100,batch_size = 4, validation_data=[test,label])
history = NN_model.fit(X_train,Y_train,epochs=100,batch_size = 5,validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
Y_pred = NN_model.predict(X_test)
plt.plot(history.history['loss'],label = 'loss')
plt.plot(history.history['val_loss'],label = 'val_loss')
plt.legend()
print(mean_squared_error(Y_pred, Y_test))

In [ ]:
data_test = pd.read_csv("E_Vo_dopedPerovskites_interpolated.csv")
df_test= pd.DataFrame(data_test)

In [ ]:
df_test = df_test[['Radius A [ang]', 'A-Density','A-AtomicNumber','A-SpecificHeat','B-AtomicMass',
                   'Vacancy energy [eV/O atom]']]

In [ ]:
df_test.head(15)

In [ ]:
test = df_test.values[:, 0:-1]
label = df_test.values[:,-1]

In [ ]:
scaler = StandardScaler()
test = scaler.fit_transform(test)

In [ ]:
test_pred = NN_model.predict(test)

In [ ]:
mean_squared_error(label,test_pred)

In [ ]:
test_pred

In [ ]:
import matplotlib.pyplot as plt
Y_pred = NN_model.predict(test)
plt.plot(history.history['loss'],label = 'loss')
plt.plot(history.history['val_loss'],label = 'val_loss')
plt.legend()
print(mean_squared_error(Y_pred, label))